In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import torch
import torchvision
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, WeightedRandomSampler, Subset
import numpy as np
import time


In [34]:
# Path to the folder containing images
data_dir = '/content/drive/MyDrive/JPEGImages'

# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loading datasets
full_dataset = datasets.ImageFolder(root=data_dir)

# Splitting dataset
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Apply appropriate transforms to each dataset using Subset and setting transform post split
train_dataset = Subset(full_dataset, train_dataset.indices)
test_dataset = Subset(full_dataset, test_dataset.indices)

# Manually apply transforms by wrapping the original datasets
train_dataset.dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)
test_dataset.dataset = datasets.ImageFolder(root=data_dir, transform=test_transforms)

In [35]:
# Calculate class weights using tensor operations
train_labels = torch.tensor([train_dataset.dataset.targets[i] for i in train_dataset.indices])
class_counts = torch.bincount(train_labels, minlength=50)
class_weights = 1.0 / (class_counts.float() + 1e-6)
samples_weights = class_weights[train_labels]

# Sampler
sampler = WeightedRandomSampler(samples_weights, len(samples_weights), replacement=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=8, pin_memory=True)



In [36]:
# Model definition
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, 50)
)

# Move model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

In [37]:
# Training loop
for epoch in range(10):
    start_time = time.time()  # Start time of the epoch
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        if (i + 1) % 20 == 0:
            print(f'Epoch {epoch + 1}, Step {i + 1}, Loss: {running_loss / 20:.4f}')
            running_loss = 0.0

    end_time = time.time()  # End time of the epoch
    epoch_time = end_time - start_time  # Calculate the total time for the epoch
    print(f'Epoch {epoch + 1} completed in {epoch_time:.2f} seconds')




Epoch 1, Step 20, Loss: 3.1625
Epoch 1, Step 40, Loss: 2.6489
Epoch 1, Step 60, Loss: 2.3343
Epoch 1, Step 80, Loss: 2.0091
Epoch 1, Step 100, Loss: 2.0103
Epoch 1, Step 120, Loss: 1.9547
Epoch 1 completed in 29.67 seconds
Epoch 2, Step 20, Loss: 1.7472
Epoch 2, Step 40, Loss: 1.6515
Epoch 2, Step 60, Loss: 1.5972
Epoch 2, Step 80, Loss: 1.5225
Epoch 2, Step 100, Loss: 1.4502
Epoch 2, Step 120, Loss: 1.4763
Epoch 2 completed in 29.32 seconds
Epoch 3, Step 20, Loss: 1.4679
Epoch 3, Step 40, Loss: 1.3802
Epoch 3, Step 60, Loss: 1.3436
Epoch 3, Step 80, Loss: 1.2996
Epoch 3, Step 100, Loss: 1.1877
Epoch 3, Step 120, Loss: 1.2475
Epoch 3 completed in 29.26 seconds
Epoch 4, Step 20, Loss: 1.1979
Epoch 4, Step 40, Loss: 1.0566
Epoch 4, Step 60, Loss: 1.1205
Epoch 4, Step 80, Loss: 0.9928
Epoch 4, Step 100, Loss: 1.0228
Epoch 4, Step 120, Loss: 1.0305
Epoch 4 completed in 29.31 seconds
Epoch 5, Step 20, Loss: 0.9772
Epoch 5, Step 40, Loss: 0.9228
Epoch 5, Step 60, Loss: 0.8997
Epoch 5, Step 8

In [38]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}% on the test dataset')



Accuracy: 70.08% on the test dataset
